In [94]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import datetime as dt
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns

In [100]:
os.chdir('C:\Work\WORK_PACKAGE\Demand_forecasting\github\DeepAR-pytorch\My_model\Rental_Fleet_Demand_Forecast\data\station_level\outflow_data')

UnmetDemand_df_station_lvl = pd.read_parquet('UnmetDemand_df.parquet')

# identify if any time series is all 0
for c in UnmetDemand_df_station_lvl.columns:
    if UnmetDemand_df_station_lvl[c].nunique() == 1:
        print('station with all zeros: ',c)

UnmetDemand_df_station_lvl

,4,5,6,7,8,9,10,11,12,13,...,380,381,382,383,384,385,386,387,388,389
2021-09-24 00:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-09-24 01:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-09-24 02:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-09-24 03:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-09-24 04:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-23 15:00:00,1,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2021-12-23 16:00:00,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-12-23 17:00:00,1,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2021-12-23 18:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [101]:
UnmetDemand_df_station_lvl.index.name = 'datetime'
UnmetDemand_df_station_lvl.index.name

'datetime'

In [102]:
UnmetDemand_df_station_lvl['4'].values

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [44]:
UnmetDemand_df_vstack = pd.DataFrame(columns=['datetime','station_id','unmet_demand'])
# UnmetDemand_df_vstack.index.name = 'datetime'

for c in UnmetDemand_df_station_lvl.columns:
    temp_df = pd.DataFrame(columns=['station_id','unmet_demand'])
    temp_df['station_id'] = np.repeat(int(c),UnmetDemand_df_station_lvl.shape[0])
    temp_df['unmet_demand'] = UnmetDemand_df_station_lvl[c].values
    temp_df['datetime'] = UnmetDemand_df_station_lvl.index
    #temp_df.index = UnmetDemand_df_station_lvl.index
    UnmetDemand_df_vstack = pd.concat([UnmetDemand_df_vstack, temp_df],axis=0)

UnmetDemand_df_vstack['unmet_demand'].hist()
UnmetDemand_df_vstack

,datetime,station_id,unmet_demand
0,2021-09-24 00:00:00,4,0
1,2021-09-24 01:00:00,4,0
2,2021-09-24 02:00:00,4,0
3,2021-09-24 03:00:00,4,0
4,2021-09-24 04:00:00,4,0
...,...,...,...
2175,2021-12-23 15:00:00,389,0
2176,2021-12-23 16:00:00,389,0
2177,2021-12-23 17:00:00,389,0
2178,2021-12-23 18:00:00,389,0


In [86]:
from pandas_profiling.visualisation.plot import timeseries_heatmap

#timeseries_heatmap(dataframe=UnmetDemand_df_vstack,entity_column='station_id',sortby='datetime',figsize=(555,55),max_entities=10)

#UnmetDemand_df_station_lvl[UnmetDemand_df_station_lvl.columns[:2]].plot(figsize=(25,25),marker='*')
#sns.heatmap(UnmetDemand_df_station_lvl, annot=True)


#plt.show()

In [ ]:
# PANDAS profiling
profile = ProfileReport(UnmetDemand_df_vstack, tsmode=True, sortby="datetime")
os.chdir('C:\Work\WORK_PACKAGE\Demand_forecasting\github\DeepAR-pytorch\My_model\Rental_Fleet_Demand_Forecast\docs\\read_me\pandas_data_profile')
#profile.to_file('profile_report.html')


# Return the profile per station
for group in UnmetDemand_df_vstack.groupby("station_id"):
    #Running 1 profile per station
    #print(group[1])
    profile = ProfileReport(
        group[1],
        tsmode=True,
        sortby="datetime",
        # title=f"Air Quality profiling - Site Num: {group[0]}",
    )

    profile.to_file(f"Ts_Profile_{group[0]}.html")

In [5]:
# compare to met demand at station level to check that it is really overflowing demand and not due to the
# person cancelling

os.chdir('C:\Work\WORK_PACKAGE\Demand_forecasting\github\DeepAR-pytorch\My_model\Rental_Fleet_Demand_Forecast\data\station_level\outflow_data')
MetDemand_df_station_lvl = pd.read_csv('station_outflow.csv')
MetDemand_df_station_lvl

# check cross correlation and ensure that lags close to zero have
# highest correlation




# plot a scatter plot of peak times from met and unmet demand time series
# check that the line is 45 degrees



,4,5,6,7,8,9,10,11,12,13,...,380,381,382,383,384,385,386,387,388,389
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13018,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13019,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13020,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13021,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
UnmetDemand_df_cluster_lvl = 
UnmetDemand_df_region_lvl = 
UnmetDemand_df_country_lvl = 